# Clonamos el repositorio para obtener los dataSet

In [ ]:
!git clone https://github.com/joanby/tensorflow.git

# Damos acceso a nuestro Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Test it

In [ ]:
!ls '/content/drive/My Drive' 

# Google colab tools

In [ ]:
from google.colab import files # Para manejar los archivos y, por ejemplo, exportar a su navegador
import glob # Para manejar los archivos y, por ejemplo, exportar a su navegador
from google.colab import drive # Montar tu Google drive

##Especificando la versión de TensorFlow

Ejecutando "importar tensorflow" importará la versión por defecto (actualmente 2.x). Puedes usar la 1.x ejecutando una celda con la "versión mágica de tensorflow" **antes de ejecutar "importar tensorflow".

### Si no funciona hacer el pip install


In [ ]:
#!pip install tensorflow==1.14
%tensorflow_version 1.x

# Importar Tensorflow

In [ ]:
import tensorflow as tf
print(tf.__version__)
import matplotlib.pyplot as plt

In [ ]:
session = tf.Session()

# Deep Dream

In [ ]:
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output, Image, display
import numpy as np
import PIL.Image
import tensorflow as tf
from io import BytesIO
import urllib.request
import zipfile

In [ ]:
graph = tf.Graph()
session = tf.InteractiveSession(graph=graph)

In [ ]:
model_url = "https://storage.googleapis.com/download.tensorflow.org/models/inception5h.zip"
data_dir = './datasets/model'
if not os.path.isdir(data_dir):
    os.makedirs(data_dir)

target_file = os.path.join(data_dir, 'inception5h.zip')
if not os.path.isfile(target_file):
    print("inception5h.zip no descargado, vamos a bajarlo (Tamaño estimado =~ 55 Mb)")
    file, headers = urllib.request.urlretrieve(model_url, target_file)

with zipfile.ZipFile(target_file,"r") as zip_ref:
    zip_ref.extractall(data_dir)

In [ ]:
filename = os.path.join(data_dir, 'tensorflow_inception_graph.pb')
model_nn = filename
with tf.gfile.FastGFile(model_nn, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [ ]:
t_input = tf.placeholder(np.float32, name='input')
imagenet_mean = 117.0
t_preprocessed = tf.expand_dims(t_input-imagenet_mean,0)
tf.import_graph_def(graph_def, {'input':t_preprocessed})

In [ ]:
layers = [op.name for op in graph.get_operations() if op.type=="Conv2D" and "import/" in op.name]

In [ ]:
layers

In [ ]:
feature_num = [int(graph.get_tensor_by_name(name+':0').get_shape()[-1]) for name in layers]

In [ ]:
feature_num

In [ ]:
layer = "mixed4d_3x3_bottleneck_pre_relu"
channel = 139
im_noise = np.random.uniform(size = (224,224,3))+100.0

In [ ]:
def show_array(a, fmt = "jpeg"):
  a = np.uint8(np.clip(a,0,1)*255)
  f = BytesIO()
  PIL.Image.fromarray(a).save(f, fmt)
  plt.imshow(a)

In [ ]:
def T(layer):
    return graph.get_tensor_by_name("import/%s:0"%layer)

In [ ]:
def tf_func(*argtypes):
    placeholders = list(map(tf.placeholder, argtypes))
    def wrap(f):
        out = f(*placeholders)
        def wrapper(*args, **kw):
            return out.eval(dict(zip(placeholders, args)), session=kw.get('session'))
        return wrapper
    return wrap

In [ ]:
def resize(img, size):
    img = tf.expand_dims(img,0)
    return tf.image.resize_bilinear(img, size)[0,:,:,:]

In [ ]:
def calc_grad_tiled(img, t_grad, tile_size=512):
    sz = tile_size
    h,w = img.shape[:2]
    sx, sy = np.random.randint(sz,size = 2)
    img_shift = np.roll(np.roll(img,sx,1),sy,0)
    grad = np.zeros_like(img)
    for y in range(0, max(h-sz//2, sz), sz):
        for x in range(0, max(w-sz//2, sz), sz):
            sub = img_shift[y:y+sz, x:x+sz]
            g = session.run(t_grad, {t_input:sub})
            grad[y:y+sz, x:x+sz] = g
    return np.roll(np.roll(grad, -sx, 1), -sy,0)

In [ ]:
def render_deepdream(t_obj, img0=im_noise, iter_n = 10, step = 1.5, octave_n = 4, octave_scale = 1.4):
    t_score = tf.reduce_mean(t_obj)
    t_grad = tf.gradients(t_score, t_input)[0]
    img = img0
    octaves = []
    for i in range(octave_n-1):
        hw = img.shape[:2]
        lo = resize(img, np.int32(np.float32(img.shape[:2])/octave_scale))
        hi = img-resize(lo, hw)
        img = lo
        octaves.append(hi)
    
    for octave in range(octave_n):
        if octave > 0:
            hi = octaves[-octave]#empezamos desde el último octave la reconstrucción
            img = resize(img, hi.shape[:2]) + hi
        for i in range(iter_n):
            g = calc_grad_tiled(img, t_grad)
            img += g*(step / (np.abs(g).mean()+1e-7))
            print(".", end=" ")
        show_array(img/255.0)

In [ ]:
resize = tf_func(np.float32, np.int32)(resize)

In [ ]:
img0 = PIL.Image.open("/content/tensorflow/datasets/stylenet/original_image.jpg")
img0 = np.float32(img0)

In [ ]:
show_array(img0/255)

### 4) ¡Soñemos profundamente!

Puedes ajustar los deslizadores para cambiar la fuerza del sueño profundo, y sobre cuántas escalas se aplica.

In [ ]:
octave_n = 4 #@param {type:"slider", max: 10}
octave_scale = 1.4 #@param {type:"number"}
iter_n = 10 #@param {type:"slider", max: 50}
strength = 200 #@param {type:"slider", max: 1000}
layer = "mixed4c"  #@param ["mixed3a", "mixed3b", "mixed4a", "mixed4c", "mixed5a"]

final = render_deepdream(tf.square(T(layer)), img0)

### 5) Las neuronas individuales

También podemos tratar de optimizar no contra una capa entera sino contra la actividad de una sola neurona:

In [ ]:
feature_channel = 139 #@param {type:"slider", max: 512}
layer = "mixed4d_3x3_bottleneck_pre_relu"  #@param ["mixed4d_3x3_bottleneck_pre_relu", "mixed3a", "mixed3b", "mixed4a", "mixed4c", "mixed5a"]
if feature_channel >= T(layer).shape[3]:
  print("Feature channel exceeds size of layer ", layer, " feature space. ")
  print("Choose a smaller channel number.")
else:
  render_deepdream(T(layer)[:,:,:,feature_channel], img0)